# Understanding Transformers using PyTorch
https://www.geeksforgeeks.org/deep-learning/transformer-using-pytorch/

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy